# viresclient API

> Author: Luca Mariani
>
> Abstract: Describe the main classes and methods defined in the viresclient package

## Introduction

The `viresclient` Python package allows to connect to the VirES server to download [Swarm](https://earth.esa.int/web/guest/missions/esa-operational-eo-missions/swarm) data and data calculated using magnetic models.

Documentation:

- https://viresclient.readthedocs.io/

<a id="top"/>

## Contents
- [Access token configuration](#access_token_configuration)
- [Send requests to the server - SwarmRequest](#SwarmRequest)
    - [Get the available collections](#SwarmRequest.available_collections)
    - [Get the available measurements](#SwarmRequest.available_measurements)
    - [Get the available auxiliaries](#SwarmRequest.available_auxiliaries)
    - [Get the available magnetic models](#SwarmRequest.available_models)
    - [Get information about one or mode models](#SwarmRequest.get_model_info)
    - [Get the orbit number](#SwarmRequest.get_orbit_number)
    - [Get times for orbits](#SwarmRequest.get_times_for_orbits)
    - [Set collections](#SwarmRequest.set_collection)
    - [Set products](#SwarmRequest.set_products)
    - [Set/clear filters](#SwarmRequest.set_range_filter)
    - [Send request to the server](#SwarmRequest.get_between)
- [Handle downloaded data - ReturnedData](#ReturnedData)
    - [Get the list of source data](#ReturnedData.sources)
    - [ReturnedData contents](#ReturnedData.contents)
    - [Get type of downloaded data files](#ReturnedData.filetype)
    - [Get list of magnetic models used during calculations](#ReturnedData.magnetic_models)
    - [Get list of filters applied to the request](#ReturnedData.range_filters)
    - [Convert ReturnedData to Pandas DataFrame](#ReturnedData.as_dataframe)
    - [Convert ReturnedData to xarray Dataset](#ReturnedData.as_xarray)
    - [Save downloaded data to a file](#ReturnedData.to_file)
    - [Save downloaded data to multiple files](#ReturnedData.to_files)
- [Handle downloaded temporary file - ReturnedDataFile](#ReturnedDataFile)
    - [Get NamedTemporaryFile associated to a ReturnedDataFile](#ReturnedDataFile._file)
    - [Get type of the downloaded data file](#ReturnedDataFile.filetype)
    - [Convert ReturnedDataFile to Pandas DataFrame](#ReturnedDataFile.as_dataframe)
    - [Convert ReturnedDataFile to xarray Dataset](#ReturnedDataFile.as_xarray)
    - [Save ReturnedDataFile object to a file](#ReturnedDataFile.to_file)
- [Handle viresclient configuration - ClientConfig](#ClientConfig)
    - [Get path of the configuration file](#ClientConfig.path)
    - [Get or set the default URL](#ClientConfig.default_url)
    - [Set site configuration](#ClientConfig.set_site_config)
    - [Get site configuration](#ClientConfig.get_site_config)
    - [Save configuration](#ClientConfig.save)
- [Upload data to the server - DataUpload](#DataUpload)
    - [Upload a file to the server](#DataUpload.post)
    - [Get the identifier(s) of the uploaded file(s)](#DataUpload.ids)
    - [Get info about the uploaded file](#DataUpload.get)
    - [Set constant parameters to the uploaded file](#DataUpload.set_constant_parameters)
    - [Get constant parameters](#DataUpload.get_constant_parameters)
    - [Delete a specific uploaded file](#DataUpload.delete)
    - [Delete the uploaded files](#DataUpload.clear)

[TOP](#top)

<a id="access_token_configuration"/>

## Access token configuration

Before using the client, you need to set the access token. This can be done using the `set_token()` function:

```python
set_token(url='https://vires.services/ows', token=None, set_default=False)
```

**Parameters**:

- **url** (*str*, optional): server URL (default value: `https://vires.services/ows`).
- **token** (*str*, optional): token string obtained from the VirES access token management page (https://vires.services/accounts/tokens/). If this parameter is not set, the user is prompted to enter its value interactively.
- **set_default** (*bool*, optional): if `True`, the server identified by *url* is configured as default server (default `False`).

Since the *url* parameter is by default set to the correct value you don't need to specify it. You can avoid also to specify the *token* parameter because this value will be asked interactively. Moreover, setting `set_default=True` you can configure this URL as default in the configuration file and avoid to specify it while sending requests to the server.

First, the function must be imported from the `viresclient` package:

In [ ]:
from viresclient import set_token

Now it can be executed:

In [ ]:
set_token(set_default=True)

[TOP](#top)

<a id="SwarmRequest"/>

## Send requests to the server - SwarmRequest

The `SwarmRequest` object allows to create a request to the server and to download data according to the input parameters.

```python
class SwarmRequest(url=None, username=None, password=None, token=None, config=None, logging_level='NO_LOGGING')
```

**Parameters**:

- **url** (*str*, optional): if not provided, the request will be sent to the default URL set in the `~/.viresclient.ini` configuration file.
- **username** (*str*, optional): username. The usage of username and password is deprecated and will be removed in future releases.
- **password** (*str*, optional): password. The usage of username and password is deprecated and will be removed in future releases.
- **token** (*str*, optional): token string obtained from the VirES access token management page (https://vires.services/accounts/tokens/). If this parameter is not specified and it is not set in the configuration file, the user is prompted to enter its value interactively.
- **config** (*str* or *ClientConfig*, optional): viresclient configuration. By default, it is read from `~/.viresclient.ini`.
- **logging_level** (*str*, optional): set the logging level. Allowed values are: `DEBUG`, `INFO`, `WARNING`, `ERROR`, `NO_LOGGING` (default).

Let's import the class from the `viresclient` package:

In [ ]:
from viresclient import SwarmRequest

After configuring the access token using the `set_token()` function (see [Access token configuration](#access_token_configuration)) , the `SwarmRequest` object can be created as follows:

In [ ]:
request = SwarmRequest()

The access token for the `https://vires.services/ows` default URL configured in the `~/.viresclient.ini` file is automatically retrieved:

```ini
[https://vires.services/ows]
token = <token>

[default]
url = https://vires.services/ows
```

If the default server URL is not configured (i.e. the "default" section is not present), you must specify the server URL:

In [ ]:
request = SwarmRequest('https://vires.services/ows')

`SwarmRequest` object has the following methods:

- `SwarmRequest.available_collections()`
- `SwarmRequest.available_measurements()`
- `SwarmRequest.available_auxiliaries()`
- `SwarmRequest.available_models()`
- `SwarmRequest.get_model_info()`
- `SwarmRequest.get_orbit_number()`
- `SwarmRequest.get_times_for_orbits()`
- `SwarmRequest.set_collection()`
- `SwarmRequest.set_products()`
- `SwarmRequest.set_range_filter()`
- `SwarmRequest.clear_range_filter()`
- `SwarmRequest.get_between()`



[TOP](#top)

<a id="SwarmRequest.available_collections"/>

### Get the available collections

Swarm data are organized in *collections*. Each collection is related to a Swarm file type (e.g. collection *SW_OPER_MAGA_LR_1B* is related to file type *MAGA_LR_1B*). The list of the available collections are provided invoking the `SwarmRequest.available_collections()` method.

```python
SwarmRequest.available_collections(details=True)
```

**Parameters**:

- **details** (*bool*, optional): if `True` (default), the method prints the list of all the available collections and the related details. If `False`, it returns the available collections as a *list*.

**Example**: print the list of the available collections and their details:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get the available collections
request.available_collections()

**Example**: get the list of available collections without the details:

In [ ]:
request.available_collections(details=False)

[TOP](#top)

<a id="SwarmRequest.available_measurements"/>

### Get the available measurements

It is possible to get the available measurements using the `SwarmRequest.available_measurements()` method:

```python
SwarmRequest.available_measurements(collection=None)
```

**Parameters**:

- **collection** (*str*, optional). If indicated, returns the available measurements for this collection as a *list*. It can be set to: `MAG`, `EFI`, `IBI`, `TEC`, `FAC`, `EEF`, `IPD` or one of the collections returned by the *available_collections* method. If not indicated, it returns the available measurements for all the collections as a *dict*.

**Example**: get the list of measurements for the `MAG` collections:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get the available measurements
request.available_measurements('MAG')

Setting `collection=SW_OPER_MAGA_LR_1B` we obtain the same result:

In [ ]:
request.available_measurements('SW_OPER_MAGA_LR_1B')

**Example**: get all the available measurements:

In [ ]:
request.available_measurements()

[TOP](#top)

<a id="SwarmRequest.available_auxiliaries"/>

### Get the available auxiliaries

It is possible to get the available auxiliaries using the `SwarmRequest.available_auxiliaries()` method:

```python
SwarmRequest.available_auxiliaries()
```

This method does not accept input parameters and provides the available auxiliaries as a list.

**Example**: get the list of the available auxiliaries:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get the list of available auxiliaries
request.available_auxiliaries()

[TOP](#top)

<a id="SwarmRequest.available_models"/>

### Get the available magnetic models

The available magnetic models can be obtained using the `SwarmRequest.available_models()` method:

```python
SwarmRequest.available_models(param=None, details=True, nice_output=True)
```

**Parameters**:

- **param** (*str*, optional): it can be set to one of `F`, `C`, `D`, `MCO`, `MLI`, `MMA`, `MIO` to filter all the available magnetic models.
- **details** (*bool*, optional): if it is set to `True` (default), it gets the models and their details. If `False`, it returns only the models as a *list*.
- **nice_output** (*bool*, optional): if it is set to `True` (default), it prints the models and their details. If `False`, it returns the models as a dictionary (if `details=True`) or a s a list (if `details=False`).

**Example**: get the list of all the available models and their details:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get available models
request.available_models()

**Example**: print the list of the available `F` (Fast-Track) models and their details:

In [ ]:
request.available_models('F')

**Example**: get the If the list of models of type `F` as a dictionary:

In [ ]:
request.available_models(param='F', nice_output=False)

**Example**: get the list of all the available models without their details:

In [ ]:
request.available_models(details=False)

[TOP](#top)

<a id="SwarmRequest.get_model_info"/>

### Get information about one or more models.

It is possible to get information about one or more specific models using the `SwarmRequest.get_model_info()` method:

```python
SwarmRequest.get_model_info(models=None, custom_model=None, original_response=False)
```

**Parameters**:

- **models** (*list[str]*, optional): models as a list of strings. If the list is not provided, returns information about all the available magnetic models.
- **custom_model** (*str*, optional): name of the file containing the spherical harmonics coefficients (custom model).
- **original_response** (*bool*, optional): if set to `False` (default), returns the result as a dictionary. If set to `True`, returns the result as a list of dictionaries.

**Example**: get info about all the available magnetic models:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get model info
request.get_model_info()

**Example**: get info about CHAOS-Core and CHAOS-Static:

In [ ]:
request.get_model_info(models=['CHAOS-Core', 'CHAOS-Static'])

**Example**: set `original_response=True` to get the result as a list of dictionaries:

In [ ]:
request.get_model_info(models=['CHAOS-Core', 'CHAOS-Static'], original_response=True)

**Example**: get info on a custom model by providing the name of the file containing its coefficients:

In [ ]:
# Download shc file with WGET (command line tool)
!wget "http://www.spacecenter.dk/files/magnetic-models/LCS-1/LCS-1.shc"

# Upload a .shc file and update the file name
request.get_model_info(custom_model='LCS-1.shc')

In [ ]:
# Delete downloaded file(s)
!rm LCS-1.shc*

[TOP](#top)

<a id="SwarmRequest.get_orbit_number"/>

### Get the orbit number

The `SwarmRequest.get_orbit_number()` allows to get the orbit number of a given spacecraft providing date and time:

```python
SwarmRequest.get_orbit_number(spacecraft, input_time)
```

**Parameters**:

- **spacecraft** (*str*): spacecraft identifier: `A`, `B` or `C`.
- **input_time** (*datetime.datetime* or *str*): date and time.

**Example**: get orbit numbers corresponding to date 2020-01-01 00:00:00 for the three spacecrafts:

In [ ]:
# Create the request object
request = SwarmRequest()

# Get results
for sc in ('A', 'B', 'C'):
    orbit = request.get_orbit_number(sc, '2020-01-01T00:00:00')
    print(f's/c {sc}: {orbit}')

[TOP](#top)

<a id="SwarmRequest.get_times_for_orbits"/>

### Get times for orbits

Get the time interval corresponding to a pair of orbit numbers using the `SwarmRequest.get_times_for_orbits()` method:

```python
SwarmRequest.get_times_for_orbits(spacecraft, start_orbit, end_orbit)
```

**Parameters**:

- **spacecraft** (*str*): spacecraft identifier: `A`, `B` or `C`.
- **start_orbit** (*int*): start orbit number
- **end_orbit** (*int*): end orbit number

**Example**: get time intervals corresponding to *start_orbit* 1000 and *end_orbit* 2000 for the three spacecrafts:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Set start/end orbits
start_orbit = 1000
end_orbit = 2000

# Create the request object
request = SwarmRequest()

# Get results
for sc in ('A', 'B', 'C'):
    start_date, end_date = request.get_times_for_orbits(sc, start_orbit, end_orbit)
    print(f's/c {sc}: {start_date} - {end_date}')

[TOP](#top)

<a id="SwarmRequest.set_collection"/>

### Set collections

Before sending the request to the server, you need to set the Swarm collection including the measurement(s) of interest. This can be done using the `SwarmRequest.set_collection()` method:

```python
SwarmRequest.set_collection(*args)
```

**Parameters**:

- ***args** (*str*): one or more collections (see [Get available collections](#SwarmRequest.available_collections)) as a string.

**Example**: to get data from SW_OPER_MAGA_LR_1B and SW_OPER_EFIA_LP_1B collections:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

[TOP](#top)

<a id="SwarmRequest.set_products"/>

### Set products

After setting the collection, you must set the combination of measurements and/or auxiliaries and/or magnetic model(s) data to retrieve. This can be done using the `SwarmRequest.set_products()` method:

```python
SwarmRequest.set_products(measurements=None, models=None, custom_model=None, auxiliaries=None, residuals=False, sampling_step=None)
```

**Parameters**:

- **measurements** (*list[str]*, optional): list of measurements to be downloaded. To get the list of the available measurements see [Available measurements](#SwarmRequest.available_measurements) (e.g.: `['F', 'B_NEC', 'Ne']`.).
- **models** (*list[str]*, optional): list of magnetic models. To get the list of the available models see [Available models](#SwarmRequest.available_models) (e.g.: `['CHAOS-Core', 'CHAOS-Static']`). In addition to the list, this parameters accepts also expression for the definition of magnetic models (e.g.: `'CHAOS = "CHAOS-Core" + "CHAOS-Static"'`).
- **custom_model** (*str*, optional): path to the the file containing the spherical harmonics coefficients of the custom model.
- **auxiliaries** (*list[str]*, optional): list of auxiliaries to be downloaded. To get the list of the available auxiliaries see [Available auxiliaries](#SwarmRequest.available_auxiliaries). Please note that the following parameters are always retrieved (i.e. they don't need to be specified): `Spacecraft`, `Timestamp`, `Latitude`, `Longitude`, `Radius`.
- **residuals** (*bool*, optional): if it is set to `True`, returns the residuals between measurements (specified with *measurements*) and models (specified with *models*). If it is set to `False` (default), returns measurements and models.
- **sampling_step** (*str*, optional): set the sampling step as an [ISO 8601 time interval](https://en.wikipedia.org/wiki/ISO_8601#Time_intervals). If not provided, data is returned with the original sampling.


**Example**: get measurements: `F` and `B_NEC` from `SW_OPER_MAGA_LR_1B` and `Ne` from `SW_OPER_EFIA_LP_1B`, model `CHAOS = "CHAOS-Core" + "Chaos-Static"` and auxiliary `OrbitNumber` with a sampling step of 10 seconds:

In [ ]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

# Set products
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)

[TOP](#top)

<a id="SwarmRequest.set_range_filter"/>

### Set/clear filters

Filter(s) can be applied to the requested measurements using the `SwarmRequest.set_range_filter()` method:

```python
SwarmRequest.set_range_filter(parameter=None, minimum=None, maximum=None)
```

This method allows to set filter(s) in the form: $minimum \le parameter \le maximum$.

**Parameters**:

- **parameter** (*str*, optional): parameter to be used as a filter (e.g. `Latitude`)
- **minimum** (*float*, optional): allowed minimum value
- **maximum** (*float*, optional): allowed maximum value

It is possible to apply multiple filters with consecutive calls to this method.

**Example:** to set filter: -20 <= `Longitude` <= 50 and 30 <= `Latitude` <= 70:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collection
request.set_collection('SW_OPER_MAGA_LR_1B')

# Set product
request.set_products(measurements=['F', 'B_NEC'])

# Set filters
request.set_range_filter('Longitude', -20.0, 50.0)
request.set_range_filter('Latitude', 30.0, 70.0)

Filters can be removed using the `SwarmRequest.clear_range_filter()` method:

```python
SwarmRequest.clear_range_filter()
```

In [ ]:
request.clear_range_filter()

[TOP](#top)

<a id="SwarmRequest.get_between"/>

### Send request to the server

After setting collection(s), measurements, auxiliaries and models, we are ready to send the request to the server using the `SwarmRequest.get_between()` method:

```python
SwarmRequest.get_between(start_time=None, end_time=None, filetype='cdf', asynchronous=True, show_progress=True, nrecords_limit=None, tmpdir=None)
```

**Parameters**:

- **start_time** (*datetime.datetime* or *str*, optional): lower bound of temporal interval. If provided as string, it must be compliant to [ISO-8601](https://en.wikipedia.org/wiki/ISO_8601).
- **end_time** (*datetime.datetime* or *str*, optional): upper bound of temporal interval. If provided as string, it must be compliant to [ISO-8601](https://en.wikipedia.org/wiki/ISO_8601).
- **filetype** (*str*, optional): file format. Allowed values: `csv` and `cdf` (default).
- **asynchronous** (*bool*, optional): if `True` (default), set the asynchronous processing.
- **show_progress** (*bool*, optional): if `True` (default), enable the progress bar while processing and downloading data.
- **nrecords_limit** (*int*): overrides the limit of 3456000 records.


**Example**: download data according to the following inut parameters and get the list of source data files:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T00:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

# Set products
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)

# Get data
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

Data is returned as a `ReturnedData` object:

In [ ]:
type(data)

[TOP](#top)

<a id="ReturnedData"/>

## Handle downloaded data - ReturnedData

Once the server receives the user's request, it creates the product, than this product is automatically downloaded and returned to the client as a `ReturnedData` object. Thus you don't need to create it by yourself. It is now possible to convert this data to a `pandas.DataFrame` object or to a `xarray.Dataset` object or to save it to one or more files.

`ReturnedData` object has the following attributes:

- `ReturnedData.sources`
- `ReturnedData.contents`
- `ReturnedData.file_types`
- `ReturnedData.magnetic_models`
- `ReturnedData.range_filters`

and the following methods:

- `ReturnedData.as_dataframe()`
- `ReturnedData.as_xarray()`
- `ReturnedData.to_file()`
- `ReturnedData.to_files()`


[TOP](#top)

<a id="ReturnedData.sources"/>

### Get the list of source data

This attribute contains the list of source data files from which the values have been extracted.

**Example**: download data according to the following inut parameters and get the list of source data files:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T00:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Get list of sources
data.sources

This is the list of the files from which the measurement and auxiliaries values and the magnetic models values have been retrieved.

[TOP](#top)

<a id="ReturnedData.contents"/>

### ReturnedData contents

Downloaded data is saved to one or more temporary files represente as `ReturnedDataFile` objects. The `ReturnedData.contents` attribute contains the list of these objects.

**Example**: download data according to the following inut parameters and get the `ReturnedData` contents:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T00:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Get the ReturnedData contents
data.contents

[TOP](#top)

<a id="ReturnedData.filetype"/>

### Get type of downloaded data files

This attribute contains the type of downloaded files (i.e. `cdf` or `csv`).

**Example**: download data according to the following inut parameters and get the `ReturnedData` file type:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T00:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

# Get ReturnedData file type
data.filetype

[TOP](#top)

<a id="ReturnedData.magnetic_models"/>

### Get list of magnetic models used during calculations

This attribute contains the list of the magnetic models used during calculations.

**Example**: get F and B_NEC from SW_OPER_MAGA_LR_1B, Ne from SW_OPER_EFIA_LP_1B, model CHAOS = "CHAOS-Core" + "Chaos-Static" and auxiliary OrbitNumber with a sampling step of 10 seconds, between 2019-10-01T00:00:00 and 2019-10-01T01:00:00 in CDF format and get the list of magnetic models used during calculations:

**Example**: download data according to the following inut parameters and get the list of magnetic models:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Get magnetic models
data.magnetic_models

[TOP](#top)

<a id="ReturnedData.range_filters"/>

### Get list of filters applied to the request

This attribute contains the list of applied filters

**Example**: download data according to the following inut parameters and get the list of the applied filters:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-02T00:00:00]
- file format: CDF
- filters:
    - Longitude: $[-20.0, 50.0]$
    - Latitude: $[30.0, 70.0]$

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
request.set_range_filter('Longitude', -20.0, 50.0)
request.set_range_filter('Latitude', 30.0, 70.0)
data = request.get_between('2019-10-01T00:00:00', '2019-10-02T00:00:00')

In [ ]:
# Get list of applied filters
data.range_filters

[TOP](#top)

<a id="ReturnedData.as_dataframe"/>

### Convert ReturnedData to Pandas DataFrame

Data downloaded from the server can be converted to a `pandas.DataFrame` object. This is a general 2D labeled, size-mutable tabular structure with potentially heterogeneously-typed column allowing the user to directly access data. For more information:

- https://pandas.pydata.org/pandas-docs/stable/
- https://pandas.pydata.org/pandas-docs/stable/reference/frame.html

This conversion can be obtained with the `ReturnedData.as_dataframe()` method:

```python
ReturnedData.as_dataframe(expand=False)
```

**Parameters**:

- **expand** (*bool*, optional): If set to `False` (default), the vector parameters are represented as arrays (i.e. all the vector components in the same column). If this parameter is stet to `True`, the vector parameters are expanded (i.e. each component in a separate column). 

**Example**: download data according to the following inut parameters and convert the `ReturnedData` object to a pandas `DataFrame`:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Convert ReturnedData to pandas DataFrame
df = data.as_dataframe()

You can visualize the first 5 records using the `DataFrame.head()` method:

In [ ]:
df.head()

Setting `expand=True`:

In [ ]:
df = data.as_dataframe(expand=True)

each vector component is in a separate column:

In [ ]:
df.head()

[TOP](#top)

<a id="ReturnedData.as_xarray"/>

### Convert ReturnedData to xarray Dataset

Data downloaded from the server can be converted to a `xarray.Dataset` object. This is a multi-dimentional array allowing the user to directly access data. For more information:

- http://xarray.pydata.org/en/stable/
- http://xarray.pydata.org/en/stable/data-structures.html#dataset

This conversion can be obtained with the `ReturnedData.as_xarray()`.

**Example**: download data according to the following inut parameters and convert the `ReturnedData` object to an xarray `Dataset`:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Convert ReturnedData to pandas DataFrame
ds = data.as_xarray()

You can se how the xarray `Dataset` is represented:

In [ ]:
ds

[TOP](#top)

<a id="ReturnedData.to_file"/>

### Save downloaded data to a file

Data downloaded from the server can be saved to a file using the `ReturnedData.to_file()` method:

```python
ReturnedData.to_file(path, overwrite=False)
```


**Parameters**:

- **path** (*str*): output file path.
- **overwrite** (*bool*, optional):if `True` allows to overwrite the file if already present at *path*.


**Example**: download data according to the following inut parameters and save the `ReturnedData` object to a file:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

**Note**: this method can be used only if the amount of downloaded data is small (i.e. if the request is not split between multiple requests).

**Example**: download data according to the following inut parameters and save the `ReturnedData` object to a file:
- measurements: U_orbit, Ne, Te, Vs
- magnetic models: none
- auxiliaries: none
- sampling step: default
- time interval: [2019-10-01T00:00:00, 2019-11-01T00:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['U_orbit', 'Ne', 'Te', 'Vs'],
)
data = request.get_between('2019-10-01T00:00:00', '2019-11-01T00:00:00')

The request is split between multiple requests. Try to execute the cell below:

In [ ]:
data.to_file('huge.cdf', True)

In [ ]:
# Remove saved files (if any)
!rm *.cdf

[TOP](#top)

<a id="ReturnedData.to_files"/>

### Save downloaded data to multiple files

Data downloaded from the server can be saved to one or more files using the `ReturnedData.to_files()` method:

```python
ReturnedData.to_files(paths, overwrite=False)
```

**Parameters**:

- **paths** (*list[str]*): output files path as a list of strings.
- **overwrite** (*bool*, optional):if `True` allows to overwrite the file if already present at *path*.


**Example**: download data according to the following inut parameters and save the `ReturnedData` object a file using the `ReturnedData.to_files()` method:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Save ReturnedData to a file
data.to_files(['out.cdf'], overwrite=True)

This method is very useful in case the request has been split between multiple requests

**Example**: download data according to the following inut parameters and save the `ReturnedData` object to a files:
- measurements: U_orbit, Ne, Te, Vs
- magnetic models: none
- auxiliaries: none
- sampling step: default
- time interval: [2019-10-01T00:00:00, 2019-11-01T00:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['U_orbit', 'Ne', 'Te', 'Vs'],
)
data = request.get_between('2019-10-01T00:00:00', '2019-11-01T00:00:00')

In [ ]:
# Save ReturnedData to files
data.to_files(['first.cdf', 'second.cdf'], overwrite=True)

**Note**: the number of files to be specified must be equal to the number of files indicated by the `ReturnedData.contents` attribute. In the above case:

In [ ]:
data.contents

In [ ]:
# Remove saved files (if any)
!rm *.cdf

[TOP](#top)

<a id="ReturnedDataFile"/>

## Handle downloaded temporary data file - ReturnedDataFile

This object holds the file downloaded from the server. Even if data has not ben saved to a file with the `ReturnedData.to_files()`, it is stored in a temporary file and automatically deleted when not needed anymore. As indicated in the "[ReturnedData contents](#ReturnedData.contents)" section, you can get the list of the returned data files using the `ReturnedData.contents` attribute. Thus, you don't need to create this object by yourself.

**Note**: the description of this object has been included for completeness only. You won't need to use this object and its methods directly. To handle the downloaded data is preferable to use the [ReturnedData](#ReturnedData) object. 

`ReturnedDataFile` object has the following attributes:

- `ReturnedDataFile._file`
- `ReturnedDataFile.filetype`

and the following methods:

- `ReturnedDataFile.as_dataframe()`
- `ReturnedDataFile.as_xarray()`
- `ReturnedDataFile.to_file()`

[TOP](#top)

<a id="ReturnedDataFile._file"/>

### Get the NamedTemporaryFile associated to a ReturnedDataFile

The `NamedTemporaryFile` corresponding to the `ReturnedDataFile` is contained in the `ReturnedDataFile._file` attribute.

**Example**: download data according to the following inut parameters and get the NamedTemporaryFiles objects associated to ReturnedDataFiles:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Get ReturnedData contents
data.contents

`ReturnedData` contains only one `ReturnedDataFile`. Let's get the associated `NamedTemporaryFile`:

In [ ]:
data.contents[0]._file

`tempfile.NamedTemporaryFile` is part of the Python standard library. For more information see: https://docs.python.org/3/library/tempfile.html#tempfile.NamedTemporaryFile

[TOP](#top)

<a id="ReturnedDataFile.filetype"/>

### Get type of the downloaded data file

This attribute contains the type of downloaded files (i.e. `cdf`, `csv` or `nc`)

**Example**: download data according to the following inut parameters and get the file type of the `ReturnedDataFile`:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Get ReturnedData contents
data.contents

In [ ]:
# Get file type
data.contents[0].filetype

[TOP](#top)

<a id="ReturnedDataFile.as_dataframe"/>

### Convert ReturnedDataFile to Pandas Dataframe

As for the `ReturnedData` object (see [Convert ReturnedData to Pandas DataFrame](#ReturnedData.as_dataframe)), the `ReturnedDataFile` object can be converted to a Pandas DataFrame using the `ReturnedDataFile.as_dataframe()` method:

```python
ReturnedDataFile.as_dataframe(expand=False)
```

**Parameters**:

- **expand** (*bool*, optional): If set to `False` (default), the vector parameters are represented as arrays (i.e. all the vector components in the same column). If this parameter is stet to `True`, the vector parameters are expanded (i.e. each component in a separate column). 

[TOP](#top)

<a id="ReturnedDataFile.as_xarray"/>

### Convert ReturnedDataFile to xarray Dataset

As for the `ReturnedData` object (see [Convert ReturnedData to xarray Dataset](#ReturnedData.as_xarray)), the `ReturnedDataFile` object can be converted to an xarray Dataset using the `ReturnedDataFile.as_xarray()` method:

```python
ReturnedDataFile.as_xarray()
```

[TOP](#top)

<a id="ReturnedDataFile.to_file"/>

### Save ReturnedDataFile object to a file

Data stored in the `ReturnedDataFile` object can be saved to a file with the `ReturnedDataFile.to_file()` method:

```python
ReturnedDataFile.to_file(path, overwrite=False)
```

**Parameters**:

- **path** (*str*): output file path.
- **overwrite** (*bool*, optional):if `True` allows to overwrite the file if already present at *path*.


**Example**: download data according to the following inut parameters and get the file type of the `ReturnedDataFile`:
- measurements: F, B_NEC, Ne
- magnetic models: CHAOS = "CHAOS-Core" + "Chaos-Static"
- auxiliaries: OrbitNumber
- sampling step: 10 seconds
- time interval: [2019-10-01T00:00:00, 2019-10-01T01:00:00]
- file format: CDF
- filters: none

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

In [ ]:
# Save the ReturnedDataFile to a file
data.contents[0].to_file('out.cdf', overwrite=True)

In [ ]:
# Remove saved files (if any)
!rm out.cdf

[TOP](#top)

<a id="ClientConfig"/>

## Handle viresclient configuration - ClientConfig

You can acces the `viresclient` configuration using the `ClientConfig` class:

```python
class viresclient.ClientConfig(path=None)
```

**Parameters**:
- **path** (*str*, optional): path of the configuration file. If not specified, the default configuration file is assumed: `~/.viresclient.ini`.

**Example**: create the `ClientConfig` object associated to the default configuration file:

In [ ]:
# Import the ClientConfig
from viresclient import ClientConfig

# Create the ClientConfig object
default = ClientConfig()

`ClientConfig` object has the following attributes:

- `ClientConfig.path`
- `ClientConfig.default_url`

and the following methods:

- `ClientConfig.set_site_config()`
- `ClientConfig.get_site_config()`
- `ClientConfig.save()`

[TOP](#top)

<a id="ClientConfig.path"/>

### Get path of the configuration file

The `ClientConfig.path` read-only attribute contains the path of the configuration file.

**Example**: create the ClientConfig object associated to the default configuration file and check its path:

In [ ]:
# Import the ClientConfig (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig

# Create the ClientConfig object
default = ClientConfig()

default.path

[TOP](#top)

<a id="ClientConfig.default_url"/>

### Get or set the default URL

The `ClientConfig.default_url` attribute contains server's default URL (i.e. the one used when `SwarmClient` class is invoked without URL).

**Example**: create the ClientConfig object associated to the default configuration file and check the default URL:

In [ ]:
# Import the ClientConfig (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig

# Create the ClientConfig object
default = ClientConfig()

default.default_url

If the default URL is not set, the attribute returns `None`.

**Example**: create a new configuration:

In [ ]:
# Import the ClientConfig (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig

# Create new configuration
newcfg = ClientConfig('newfile.ini')

# Print default URL 
print(newcfg.default_url)

Set default URL to: `https://vires.services/ows`:

In [ ]:
newcfg.default_url = 'https://vires.services/ows'

Get the updated result:

In [ ]:
# Print default URL 
print(newcfg.default_url)

[TOP](#top)

<a id="ClientConfig.set_site_config"/>

### Set site configuration

It is possible to set the configuration for a server identified by an URL with the `Client.Config.set_site_config()` method:

```python
ClientConfig.set_site_config(url, **options)
```

**Parameters**:

- **url** (*str*): server URL
- ****options** (*str*): configuration options in the form: *key*=*value* (e.g.: token='...')

**Example**: create a new configuration, set default URL to: `https://vires.services/ows` and set the access token for this URL:

In [ ]:
# Import the ClientConfig (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig

# Create new configuration
newcfg = ClientConfig('newfile.ini')

# Set default URL 
newcfg.default_url = 'https://vires.services/ows'

# Set the access token
newcfg.set_site_config(newcfg.default_url, token='mytokenstring')

[TOP](#top)

<a id="ClientConfig.get_site_config"/>

### Get site configuration

It is possible to get the configuration for a server identified by an URL with the `ClientConfig.get_site_config()` method:

```python
ClientConfig.get_site_config(url)
```

**Parameters**:

- **url** (*str*): server URL

**Example**: create a new configuration, set default URL to: `https://vires.services/ows`, set the access token for this URL and get the configuration for `https://vires.services/ows`:

In [ ]:
# Import the ClientConfig (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig

# Create new configuration
newcfg = ClientConfig('newfile.ini')

# Set default URL 
newcfg.default_url = 'https://vires.services/ows'

# Set the access token
newcfg.set_site_config(newcfg.default_url, token='mytokenstring')

# Get the configuration
newcfg.get_site_config('https://vires.services/ows')

[TOP](#top)

<a id="ClientConfig.save"/>

### Save configuration

The configuration stored in the `ClientConfig` object can be saved using the `ClientConfig.save()` method:

```python
ClientConfig.save()
```

This method saves the configuration to the path specified during the `ClientConfig` creation. You can check this value via the `ClientConfig.path` attribute.

**Example**: create a new configuration, set default URL to: `https://vires.services/ows`, set the access token for this URL and save the configuration to file:

In [ ]:
# Import the ClientConfig (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig

# Create new configuration
newcfg = ClientConfig('newfile.ini')

# Set default URL 
newcfg.default_url = 'https://vires.services/ows'

# Set the access token
newcfg.set_site_config(newcfg.default_url, token='mytokenstring')

# Save the configuration
newcfg.save()

Let's have a look to the new configuration file:

In [ ]:
!cat newfile.ini

In [ ]:
# delete newfile.ini
!rm newfile.ini

[TOP](#top)

<a id="DataUpload"/>

## Upload data to the server - DataUpload

You can upload your data to the server to view it in the VirES web interface. File format can be CDF or CSV and must be compliant to: https://github.com/ESA-VirES/VirES-Server/blob/master/vires/custom_data_format_description.md.

Data can be uploaded using the `DataUpload` object:

```python
class DataUpload(url, token, **kwargs)
```

**Parameters**:

- **url** (*str*): server URL
- **token** (*str*): access token
- ****kwargs** (*str*): additional parameters (currently not used)

`DataUpload` object has the following attributes:

- `DataUpload.ids`

and the following methods:

- `DataUpload.post()`
- `DataUpload.get()`
- `DataUpload.set_constant_parameters()`
- `DataUpload.get_constant_parameters()`

**Example**: create a `DataUpload` object for data upload to the default server. You can retrieve the default URL and the access token from the configuration, using the `ClientConfig` object:

In [ ]:
# Import DataUpload object
from viresclient import DataUpload

# Import ClientConfig object (this step can be avoided if ClientConfig has been already imported)
from viresclient import ClientConfig

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

[TOP](#top)

<a id="DataUpload.post"/>

### Upload a file to the server

You can upload a file to the server using the `DataUpload.post()` method:

```python
DataUpload.post(file, filename=None)
```

**Parameters**:

- **file** (*str*): file to be uploaded

The method returns the info about the uploaded file as a dictionary.

**Example**: upload a product to the server:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Check info about the uploaded file:

In [ ]:
info

In [ ]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.ids"/>

### Get the identifier(s) of the uploaded file(s)

You can obtain the identifiers of the uploaded files via the `DataUpload.ids` attribute as a list. Please note that currently the server accepts only one file at a time, thus the returned list will have length 1.

**Example**: upload a product to the server and get its identifier:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Get id of the uploaded file:

In [ ]:
du.ids

In [ ]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.get"/>

### Get info about the uploaded file

You can get the info of the uploaded file using the `DataUpload.get()` method:

```python
DataUpload.get(identifier=None)
```

**Parameters**:

- **identifier** (*str*, optional): identifier of the uploaded file obtained via `DataUpload.ids` attribute (see [Get the identifier(s) of the uploaded file(s)](#DataUpload.ids)) or from the info returned by the `DataUpload.post()` method (see [Upload a file to the server](#DataUpload.post)). If not provided, returns the info af all the uploaded files as a list.

**Example**: upload a product to the server and get product's info with `DataUpload.get()`:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Get info about the uploaded file:

In [ ]:
du.get(du.ids[0])

If the identifier is not provided, you will get info about all the files as a list:

In [ ]:
du.get()

In [ ]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.set_constant_parameters"/>

### Set constant parameters to the uploaded file

It is possible to set constant parameters to the uploaded file using the `DataUpload.set_constant_parameters()` method:

```python
DataUpload.set_constant_parameters(identifier, parameters, replace=False)
```

**Parameters**:

- **identifier** (*str*): file identifier.
- **parameters** (*dict*): constant parameters provided as a dictionary
- **replace** (*bool*, optional): if set to `True`, all the parameters will be replaced by the new parameters, otherwise the new parameters will update the existing ones (default behaviour).


**Example**: upload a product to the server and set constant parameters to the uploaded file:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Assign constant parameters: $param1 = 12345$ and $param2 = 34567$:

In [ ]:
du.set_constant_parameters(du.ids[0], {'param1': 12345, 'param2': 34567})

If you want to set `param1` to a new value you can update the existing set of parameters:

In [ ]:
du.set_constant_parameters(du.ids[0], {'param1': 1})

or replace the entire set of parameters:

In [ ]:
du.set_constant_parameters(du.ids[0], {'param1': 1}, replace=True)

Note that `param2` has been removed.

In [ ]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.get_constant_parameters"/>

### Get constant parameters applied to the uploaded file

It is possible to get the list of constant parameters applied to the uploaded file using the `DataUpload.get_constant_parameters()` method:

```python
DataUpload.get_constant_parameters(identifier)
```

**Parameters**:

- **identifier** (*str*): file identifier.


**Example**: upload a product to the server and get constant parameters applied to the uploaded file:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Assign constant parameters: $param1 = 12345$ and $param2 = 34567$:

In [ ]:
du.set_constant_parameters(du.ids[0], {'param1': 12345, 'param2': 34567})

Get the list of constant parameters:

In [ ]:
du.get_constant_parameters(du.ids[0])

In [ ]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.delete"/>

### Delete a specific uploaded file

You can delete a specific uploaded file using the `DataUpload.delete()` method:

```python
DataUpload.delete(identifier)
```

**Parameters**:

- **identifier** (*str*): identifier of the uploaded file obtained via `DataUpload.ids` attribute (see [Get the identifier(s) of the uploaded file(s)](#DataUpload.ids)) or from the info returned by the `DataUpload.post()` method (see [Upload a file to the server](#DataUpload.post)). If not provided, returns the info af all the uploaded files as a list.


**Example**: upload a product to the server and delete it with `DataUpload.delete()`:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Delete the uploaded product:

In [ ]:
du.delete(du.ids[0])

du.ids

In [ ]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.clear"/>

### Delete the uploaded files

You can delete *all* the uploaded files using the `DataUpload.clear()` method:

```python
DataUpload.clear()
```

**Example**: upload a product to the server and delete it with `DataUpload.clear()`:

In [ ]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Download 1 hour of MAGA_LR_1B data
request = SwarmRequest()
request.set_collection('SW_OPER_MAGA_LR_1B')
request.set_products(measurements=['F', 'B_NEC'])
data = request.get_between('2020-01-01T00:00:00', '2020-01-01T01:00:00')
data.to_file('out.cdf', overwrite=True)

In [ ]:
# Import ClientConfig and DataUpload (this step can be omitted if already executed in the previous examples)
from viresclient import ClientConfig, DataUpload

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Delete the uploaded product(s):

In [ ]:
du.clear()

du.ids